In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:18.0000 R:18.0000 rate:0.0360 aloss:0.7064 eloss:4.1539 aloss2:2.5974 exploreP:0.9982
Episode:1 meanR:22.5000 R:27.0000 rate:0.0540 aloss:0.7031 eloss:4.2166 aloss2:2.7174 exploreP:0.9956
Episode:2 meanR:23.3333 R:25.0000 rate:0.0500 aloss:0.6988 eloss:4.1235 aloss2:2.5819 exploreP:0.9931
Episode:3 meanR:20.7500 R:13.0000 rate:0.0260 aloss:0.7109 eloss:4.1038 aloss2:2.5301 exploreP:0.9918
Episode:4 meanR:20.4000 R:19.0000 rate:0.0380 aloss:0.6985 eloss:4.0859 aloss2:2.6038 exploreP:0.9900
Episode:5 meanR:23.0000 R:36.0000 rate:0.0720 aloss:0.7015 eloss:4.0721 aloss2:2.6061 exploreP:0.9864
Episode:6 meanR:21.4286 R:12.0000 rate:0.0240 aloss:0.6944 eloss:4.0545 aloss2:2.6195 exploreP:0.9853
Episode:7 meanR:21.2500 R:20.0000 rate:0.0400 aloss:0.6967 eloss:4.0358 aloss2:2.5922 exploreP:0.9833
Episode:8 meanR:20.3333 R:13.0000 rate:0.0260 aloss:0.6924 eloss:4.0289 aloss2:2.5882 exploreP:0.9820
Episode:9 meanR:21.0000 R:27.0000 rate:0.0540 aloss:0.6969 eloss:4.0078 aloss2:2.6

Episode:80 meanR:22.4691 R:20.0000 rate:0.0400 aloss:0.6872 eloss:3.2642 aloss2:2.2792 exploreP:0.8353
Episode:81 meanR:22.3902 R:16.0000 rate:0.0320 aloss:0.6838 eloss:3.2574 aloss2:2.2565 exploreP:0.8339
Episode:82 meanR:22.5663 R:37.0000 rate:0.0740 aloss:0.6879 eloss:3.2619 aloss2:2.2643 exploreP:0.8309
Episode:83 meanR:22.4881 R:16.0000 rate:0.0320 aloss:0.6878 eloss:3.2539 aloss2:2.2482 exploreP:0.8296
Episode:84 meanR:22.7176 R:42.0000 rate:0.0840 aloss:0.6842 eloss:3.2584 aloss2:2.2643 exploreP:0.8262
Episode:85 meanR:22.6512 R:17.0000 rate:0.0340 aloss:0.6860 eloss:3.2585 aloss2:2.2645 exploreP:0.8248
Episode:86 meanR:22.7241 R:29.0000 rate:0.0580 aloss:0.6853 eloss:3.2530 aloss2:2.2600 exploreP:0.8224
Episode:87 meanR:22.7955 R:29.0000 rate:0.0580 aloss:0.6844 eloss:3.2459 aloss2:2.2526 exploreP:0.8201
Episode:88 meanR:22.8989 R:32.0000 rate:0.0640 aloss:0.6870 eloss:3.2499 aloss2:2.2454 exploreP:0.8175
Episode:89 meanR:22.9000 R:23.0000 rate:0.0460 aloss:0.6814 eloss:3.2498 

Episode:159 meanR:35.7800 R:39.0000 rate:0.0780 aloss:0.6869 eloss:3.2644 aloss2:2.2629 exploreP:0.6167
Episode:160 meanR:36.1700 R:65.0000 rate:0.1300 aloss:0.7038 eloss:3.2637 aloss2:2.2683 exploreP:0.6128
Episode:161 meanR:37.2200 R:129.0000 rate:0.2580 aloss:0.6904 eloss:3.2701 aloss2:2.2617 exploreP:0.6050
Episode:162 meanR:38.5100 R:145.0000 rate:0.2900 aloss:0.6993 eloss:3.2644 aloss2:2.2555 exploreP:0.5965
Episode:163 meanR:38.9000 R:78.0000 rate:0.1560 aloss:0.7551 eloss:3.2658 aloss2:2.2441 exploreP:0.5919
Episode:164 meanR:39.0500 R:31.0000 rate:0.0620 aloss:0.6994 eloss:3.2590 aloss2:2.2594 exploreP:0.5901
Episode:165 meanR:39.0700 R:56.0000 rate:0.1120 aloss:0.7180 eloss:3.2692 aloss2:2.2447 exploreP:0.5869
Episode:166 meanR:40.1300 R:123.0000 rate:0.2460 aloss:0.6973 eloss:3.2570 aloss2:2.2517 exploreP:0.5798
Episode:167 meanR:41.2000 R:121.0000 rate:0.2420 aloss:0.7072 eloss:3.2658 aloss2:2.2319 exploreP:0.5730
Episode:168 meanR:42.0200 R:104.0000 rate:0.2080 aloss:0.696

Episode:238 meanR:134.5200 R:261.0000 rate:0.5220 aloss:0.6869 eloss:3.1036 aloss2:2.3332 exploreP:0.1891
Episode:239 meanR:138.3900 R:483.0000 rate:0.9660 aloss:0.6980 eloss:3.0980 aloss2:2.3360 exploreP:0.1807
Episode:240 meanR:139.9700 R:201.0000 rate:0.4020 aloss:0.6906 eloss:3.0994 aloss2:2.3369 exploreP:0.1773
Episode:241 meanR:142.6000 R:302.0000 rate:0.6040 aloss:0.6876 eloss:3.1031 aloss2:2.3396 exploreP:0.1723
Episode:242 meanR:144.2700 R:237.0000 rate:0.4740 aloss:0.6876 eloss:3.0950 aloss2:2.3555 exploreP:0.1685
Episode:243 meanR:146.4800 R:251.0000 rate:0.5020 aloss:0.6939 eloss:3.1035 aloss2:2.3441 exploreP:0.1646
Episode:244 meanR:147.9500 R:208.0000 rate:0.4160 aloss:0.6914 eloss:3.0912 aloss2:2.3563 exploreP:0.1614
Episode:245 meanR:149.4100 R:201.0000 rate:0.4020 aloss:0.6828 eloss:3.1059 aloss2:2.3491 exploreP:0.1584
Episode:246 meanR:151.7200 R:262.0000 rate:0.5240 aloss:0.6880 eloss:3.0935 aloss2:2.3577 exploreP:0.1546
Episode:247 meanR:153.9000 R:232.0000 rate:0.4

Episode:316 meanR:214.2600 R:234.0000 rate:0.4680 aloss:0.6632 eloss:3.0830 aloss2:2.5612 exploreP:0.0465
Episode:317 meanR:214.8000 R:156.0000 rate:0.3120 aloss:0.6676 eloss:3.0785 aloss2:2.5539 exploreP:0.0459
Episode:318 meanR:214.4300 R:160.0000 rate:0.3200 aloss:0.6647 eloss:3.0864 aloss2:2.5636 exploreP:0.0453
Episode:319 meanR:215.5500 R:173.0000 rate:0.3460 aloss:0.6589 eloss:3.0924 aloss2:2.5695 exploreP:0.0447
Episode:320 meanR:215.5800 R:246.0000 rate:0.4920 aloss:0.6459 eloss:3.0881 aloss2:2.5652 exploreP:0.0439
Episode:321 meanR:214.9200 R:183.0000 rate:0.3660 aloss:0.6433 eloss:3.0878 aloss2:2.5578 exploreP:0.0433
Episode:322 meanR:211.5300 R:161.0000 rate:0.3220 aloss:0.6636 eloss:3.0835 aloss2:2.5644 exploreP:0.0427
Episode:323 meanR:213.4600 R:216.0000 rate:0.4320 aloss:0.6494 eloss:3.0923 aloss2:2.5740 exploreP:0.0420
Episode:324 meanR:214.7400 R:180.0000 rate:0.3600 aloss:0.6679 eloss:3.0857 aloss2:2.5793 exploreP:0.0415
Episode:325 meanR:213.9900 R:159.0000 rate:0.3

...

Episode:1931 meanR:305.3400 R:155.0000 rate:0.3100 aloss:0.4137 eloss:-0.7740 aloss2:10.2975 exploreP:0.0100
Episode:1932 meanR:306.0600 R:333.0000 rate:0.6660 aloss:0.4062 eloss:-0.9681 aloss2:10.4038 exploreP:0.0100
Episode:1933 meanR:305.1300 R:112.0000 rate:0.2240 aloss:0.4017 eloss:-1.4108 aloss2:10.4844 exploreP:0.0100
Episode:1934 meanR:303.5100 R:193.0000 rate:0.3860 aloss:0.3987 eloss:-1.5208 aloss2:10.6330 exploreP:0.0100
Episode:1935 meanR:303.1400 R:221.0000 rate:0.4420 aloss:0.4056 eloss:-1.2343 aloss2:10.3670 exploreP:0.0100
Episode:1936 meanR:303.0700 R:143.0000 rate:0.2860 aloss:0.4028 eloss:-1.1040 aloss2:10.3430 exploreP:0.0100
Episode:1937 meanR:302.8600 R:173.0000 rate:0.3460 aloss:0.4056 eloss:-1.3151 aloss2:10.4940 exploreP:0.0100
Episode:1938 meanR:299.8300 R:167.0000 rate:0.3340 aloss:0.4048 eloss:-1.5378 aloss2:10.7458 exploreP:0.0100
Episode:1939 meanR:299.1600 R:169.0000 rate:0.3380 aloss:0.4050 eloss:-1.3078 aloss2:10.4131 exploreP:0.0100
Episode:1940 meanR:

Episode:2007 meanR:249.1600 R:233.0000 rate:0.4660 aloss:0.3663 eloss:-5.5712 aloss2:15.1186 exploreP:0.0100
Episode:2008 meanR:249.6200 R:318.0000 rate:0.6360 aloss:0.3678 eloss:-4.9574 aloss2:14.4609 exploreP:0.0100
Episode:2009 meanR:250.3200 R:231.0000 rate:0.4620 aloss:0.3634 eloss:-4.9435 aloss2:14.4167 exploreP:0.0100
Episode:2010 meanR:248.1200 R:209.0000 rate:0.4180 aloss:0.3624 eloss:-4.9604 aloss2:14.4367 exploreP:0.0100
Episode:2011 meanR:245.5300 R:241.0000 rate:0.4820 aloss:0.3695 eloss:-5.4624 aloss2:15.0297 exploreP:0.0100
Episode:2012 meanR:244.6000 R:158.0000 rate:0.3160 aloss:0.3597 eloss:-5.1560 aloss2:14.7095 exploreP:0.0100
Episode:2013 meanR:246.0600 R:355.0000 rate:0.7100 aloss:0.3753 eloss:-4.4821 aloss2:13.9372 exploreP:0.0100
Episode:2014 meanR:247.6900 R:384.0000 rate:0.7680 aloss:0.3693 eloss:-4.7449 aloss2:14.2168 exploreP:0.0100
Episode:2015 meanR:247.1100 R:243.0000 rate:0.4860 aloss:0.3647 eloss:-4.9834 aloss2:14.5140 exploreP:0.0100
Episode:2016 meanR:

Episode:2083 meanR:242.9100 R:415.0000 rate:0.8300 aloss:0.3614 eloss:-5.1730 aloss2:15.3409 exploreP:0.0100
Episode:2084 meanR:242.2100 R:144.0000 rate:0.2880 aloss:0.3639 eloss:-5.2173 aloss2:15.3934 exploreP:0.0100
Episode:2085 meanR:240.5900 R:164.0000 rate:0.3280 aloss:0.3627 eloss:-5.0984 aloss2:15.3331 exploreP:0.0100
Episode:2086 meanR:242.1700 R:345.0000 rate:0.6900 aloss:0.3593 eloss:-5.4214 aloss2:15.6344 exploreP:0.0100
Episode:2087 meanR:241.5300 R:151.0000 rate:0.3020 aloss:0.3619 eloss:-4.8846 aloss2:15.1061 exploreP:0.0100
Episode:2088 meanR:240.3300 R:206.0000 rate:0.4120 aloss:0.3507 eloss:-5.9352 aloss2:16.1691 exploreP:0.0100
Episode:2089 meanR:239.4800 R:294.0000 rate:0.5880 aloss:0.3637 eloss:-5.5858 aloss2:15.8208 exploreP:0.0100
Episode:2090 meanR:238.3500 R:135.0000 rate:0.2700 aloss:0.3564 eloss:-5.3952 aloss2:15.7044 exploreP:0.0100
Episode:2091 meanR:237.4000 R:185.0000 rate:0.3700 aloss:0.3702 eloss:-4.8977 aloss2:15.2465 exploreP:0.0100
Episode:2092 meanR:

Episode:2159 meanR:234.7200 R:256.0000 rate:0.5120 aloss:0.3450 eloss:-8.2281 aloss2:19.5800 exploreP:0.0100
Episode:2160 meanR:235.8100 R:279.0000 rate:0.5580 aloss:0.3489 eloss:-8.1054 aloss2:19.4653 exploreP:0.0100
Episode:2161 meanR:236.3700 R:249.0000 rate:0.4980 aloss:0.3537 eloss:-7.7877 aloss2:19.1109 exploreP:0.0100
Episode:2162 meanR:235.8900 R:195.0000 rate:0.3900 aloss:0.3395 eloss:-8.8996 aloss2:20.3671 exploreP:0.0100
Episode:2163 meanR:232.5800 R:169.0000 rate:0.3380 aloss:0.3505 eloss:-8.3716 aloss2:19.8100 exploreP:0.0100
Episode:2164 meanR:232.0400 R:164.0000 rate:0.3280 aloss:0.3448 eloss:-8.8413 aloss2:20.2828 exploreP:0.0100
Episode:2165 meanR:230.8000 R:301.0000 rate:0.6020 aloss:0.3445 eloss:-9.0441 aloss2:20.5123 exploreP:0.0100
Episode:2166 meanR:230.1300 R:204.0000 rate:0.4080 aloss:0.3497 eloss:-8.0212 aloss2:19.4607 exploreP:0.0100
Episode:2167 meanR:230.7400 R:244.0000 rate:0.4880 aloss:0.3508 eloss:-9.0661 aloss2:20.6068 exploreP:0.0100
Episode:2168 meanR:

Episode:2235 meanR:202.1600 R:150.0000 rate:0.3000 aloss:0.3707 eloss:-10.8986 aloss2:22.4737 exploreP:0.0100
Episode:2236 meanR:202.9900 R:252.0000 rate:0.5040 aloss:0.3912 eloss:-9.7504 aloss2:21.7131 exploreP:0.0100
Episode:2237 meanR:202.1900 R:235.0000 rate:0.4700 aloss:0.3735 eloss:-10.4641 aloss2:22.2050 exploreP:0.0100
Episode:2238 meanR:200.7700 R:138.0000 rate:0.2760 aloss:0.3801 eloss:-9.8531 aloss2:21.4554 exploreP:0.0100
Episode:2239 meanR:202.6400 R:243.0000 rate:0.4860 aloss:0.3826 eloss:-10.4149 aloss2:22.0302 exploreP:0.0100
Episode:2240 meanR:203.5200 R:278.0000 rate:0.5560 aloss:0.3782 eloss:-10.5853 aloss2:22.1987 exploreP:0.0100
Episode:2241 meanR:203.2800 R:185.0000 rate:0.3700 aloss:0.3791 eloss:-11.7617 aloss2:23.5957 exploreP:0.0100
Episode:2242 meanR:201.5600 R:168.0000 rate:0.3360 aloss:0.3685 eloss:-10.5908 aloss2:22.1975 exploreP:0.0100
Episode:2243 meanR:202.6100 R:202.0000 rate:0.4040 aloss:0.3780 eloss:-11.0493 aloss2:22.6555 exploreP:0.0100
Episode:2244

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
